In [1]:
import re

def parse_input(input_str):
  input_str = input_str.strip()
  
  float_or_int = r'-?(?:\d+)(?:\.\d+)?'
  frac_pattern = rf'({float_or_int})\s*\/\s*({float_or_int})'

  frac_match = re.fullmatch(frac_pattern, input_str)
  if frac_match:
    numerator, denominator = map(float, frac_match.groups())
    if denominator == 0:
      raise ZeroDivisionError("Pembagi tidak boleh nol")
    return input_str
  
  if ' ' in input_str:
    raise ValueError(f"Format tidak valid: {input_str}")
  
  if re.fullmatch(rf'{float_or_int}', input_str):
    return input_str
  
  raise ValueError(f"Format tidak valid: {input_str}")


In [2]:
def to_python_expr(coeff, degree):
  if coeff in ["0", "0.0"]:
    return None
  if '/' in coeff:
    coeff_str = f"({coeff})"
  else:
    coeff_str = coeff
  if degree == 0:
    return f"{coeff_str}"
  elif degree == 1:
    return f"{coeff_str}*x"
  else:
    return f"{coeff_str}*x**{degree}"


In [3]:
def to_pretty_string(coeff, degree):
  if coeff in ["0", "0.0"]:
    return None
  if coeff in ["1", "1.0"]:
    c = ""
  elif coeff in ["-1", [-1.0]]:
    c = "-"
  else:
    c = coeff
  
  if degree == 0:
    return f"{c}"
  elif degree == 1:
    return f"{c}x"
  else:
    return f"{c}x^{degree}"

In [4]:
def format_derivative_string(expr):
  expr = expr.strip()
  tokens = re.findall(r'[+-]?[^+-]+', expr)
  derived_terms = []

  for term in tokens:
    term = term.strip()
    if term == "":
      continue

    if 'x' not in term:
      continue

    power = 1
    power_match = re.search(r'x\*\*(\-?\d+)', term)
    if power_match:
      power = int(power_match.group(1))
    else:
      power = 1

    coef_match = re.match(r'([+-]?[\d\./\(\)]+)?\*?x', term)
    if coef_match:
      coef_str = coef_match.group(1)
      if coef_str is None or coef_str.strip() == '' or coef_str == '+':
        coef = 1
      elif coef_str == '-':
        coef = -1
      else:
        coef = eval(coef_str)
    else:
      coef = 1
    
    new_coef = power * coef
    new_power = power - 1

    sign = "-" if new_coef < 0 else "+"
    abs_coef = abs(new_coef)

    if abs_coef == 1 and new_power > 0:
      coef_part = ""
    else:
      coef_part = str(abs_coef)

    if new_power == 0:
      term_str = f"{coef_part}"
    elif new_power == 1:
      term_str = f"{coef_part}x"
    else:
      term_str = f"{coef_part}x^{new_power}"

    derived_terms.append((sign, term_str))

  if not derived_terms:
    return "0"
  
  result = ""
  for i, (sign, term) in enumerate(derived_terms):
    if i == 0:
      result += f"{'-' if sign == '-' else ''}{term}"
    else:
      result += f"{sign} {term}"
  
  return result

In [5]:
print("Masukkan koefisien untuk fungsi polinomial f(x). Masukkan 0 jika tidak ada")
inputs = []
# input user dari pangkat 4 sampai konstanta
for deg in range(4, -1, -1):
  label = f"x^{deg}" if deg > 1 else "x" if deg == 1 else "konstanta"
  raw = input(f"{label}: ")
  parsed = parse_input(raw)
  inputs.append(parsed)
  print(f"{label}: {parsed}")

# format fungsi
pretty_terms = [to_pretty_string(c, d) for c, d in zip(inputs, range(4, -1, -1))]
pretty_terms = [t for t in pretty_terms if t is not None]
pretty_expr = " + ".join(pretty_terms).replace("+-", "- ")

python_terms = [to_python_expr(c, d) for c, d in zip(inputs, range(4, -1, -1))]
python_terms = [t for t in python_terms if t is not None]
expr = " + ".join(python_terms).replace("+-", "- ")

print("\nPersamaan yang dibentuk:")
print(f"f(x) = {pretty_expr}")

print("\nEkspresi Python (f(x)):")
print(f"f(x) = {expr}")

print("\nEkspresi turunan (Python):")
derivative_expr = []
for t in python_terms:
  match = re.match(r'\(?([-\d./]+)\)?\*x\*\*(\d+)', t)
  if match:
    coef, power = match.groups()
    coef = eval(coef)
    power = int(power)
    new_coef = coef * power
    new_power = power - 1
    if new_power == 0:
      derivative_expr.append(f"{new_coef}")
    elif new_power == 1:
      derivative_expr.append(f"{new_coef}*x")
    else:
      derivative_expr.append(f"{new_coef}*x**{new_power}")
  else:
    match = re.match(r'\(?([-\d./]+)\)?\*x', t)
    if match:
      coef = eval(match.group(1))
      derivative_expr.append(str(coef))

print("f'(x) =", " + ".join(derivative_expr).replace("+-", "- "))

print("\nTurunan (format string):")
print("f'(x) =", format_derivative_string(expr))

Masukkan koefisien untuk fungsi polinomial f(x). Masukkan 0 jika tidak ada
x^4: 0
x^3: 1
x^2: 0
x: 0
konstanta: 0

Persamaan yang dibentuk:
f(x) = x^3

Ekspresi Python (f(x)):
f(x) = 1*x**3

Ekspresi turunan (Python):
f'(x) = 3*x**2

Turunan (format string):
f'(x) = 3x^2


In [6]:
# Evaluasi turunan di x = 2
x = 2
h = 0.1
f = eval(f"lambda x: {expr}")

hasil_analitik = eval(" + ".join(derivative_expr).replace("+-", "- "))
print(f"\nHasil analitik dari turunan fungsi f(x) = {hasil_analitik}")


Hasil analitik dari turunan fungsi f(x) = 12


In [7]:
def hitung_error(hasil_numerik, hasil_analitik):
  return abs( (hasil_numerik - hasil_analitik ) / hasil_analitik)

In [8]:
# Metode selisih maju
def selisih_maju(fungsi, x, h):
  return ( fungsi(x + h) - fungsi(x) ) / h

hasil_numerik = selisih_maju(f, x, h)
print("Hasil numerik dari turunan fungsi f(x) dengan metode selisih maju")
print(f"x=2, f'(2) = {hasil_numerik}")

error_persen = hitung_error(hasil_numerik, hasil_analitik) * 100
print(f"Error: {error_persen:.3f}%")

Hasil numerik dari turunan fungsi f(x) dengan metode selisih maju
x=2, f'(2) = 12.61000000000001
Error: 5.083%


In [9]:
# Metode selisih tengahan
def selisih_tengahan(fungsi, x, h):
  return ( fungsi(x + h) - fungsi(x - h) ) / (2 * h)

hasil_numerik = selisih_tengahan(f, x, h)
print("Hasil numerik dari turunan fungsi f(x) dengan metode selisih tengahan")
print(f"x=2, f'(2) = {hasil_numerik}")

error_persen = hitung_error(hasil_numerik, hasil_analitik) * 100
print(f"Error: {error_persen:.3f}%")

Hasil numerik dari turunan fungsi f(x) dengan metode selisih tengahan
x=2, f'(2) = 12.010000000000009
Error: 0.083%


In [10]:
# Metode selisih mundur
def selisih_mundur(fungsi, x, h):
  return ( fungsi(x) - fungsi(x - h) ) / h

hasil_numerik = selisih_mundur(f, x, h)
print("Hasil numerik dari turunan fungsi f(x) dengan metode selisih mundur")
print(f"x=2, f'(2) = {hasil_numerik}")

error_persen = hitung_error(hasil_numerik, hasil_analitik) * 100
print(f"Error: {error_persen:.3f}%")

Hasil numerik dari turunan fungsi f(x) dengan metode selisih mundur
x=2, f'(2) = 11.410000000000009
Error: 4.917%
